In [1]:
# Librerias, archivos y configuración

import warnings
import os
import polars as pl
from datetime import datetime
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [2]:
# Primero se unen todos los archivos de los distintos meses en uno solo

# Se hace una lista con los archivos de los distintos meses para poder llamar a la función

archivos = os.listdir('../original_pollution_data')
archivos.sort(reverse=True)

lista_meses = [e for e in archivos]

In [3]:
# Se crea el df

dfs = []

for e in lista_meses:   

        df = pl.read_csv(f'../original_pollution_data/{e}', sep=';')
    
        dfs.append(df)

df = pl.concat(dfs[:-1])

In [4]:
df.head()

PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,V06,H07,V07,H08,V08,H09,V09,H10,V10,H11,V11,H12,V12,H13,V13,H14,V14,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
i64,i64,i64,i64,str,i64,i64,i64,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str
28,79,4,1,"""28079004_1_38""",2022,12,1,1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V"""
28,79,4,1,"""28079004_1_38""",2022,12,2,1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",3.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V"""
28,79,4,1,"""28079004_1_38""",2022,12,3,1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",2.0,"""V"""
28,79,4,1,"""28079004_1_38""",2022,12,4,2.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V"""
28,79,4,1,"""28079004_1_38""",2022,12,5,1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",3.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",2.0,"""V""",2.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V""",1.0,"""V"""


In [5]:
df.shape

(425679, 56)

In [6]:
# Se ponen los nombres de las columnas en minúsculas por comodidad

df.columns = [col.lower() for col in df.columns]

In [7]:
# Es siempre la misma provincia y municipio y el punto de muestre es redundante pues se tienen datos sobre la estacion, por lo que se borran estos datos

df = df.drop(['provincia', 'municipio', 'punto_muestreo'])

In [8]:
''' Todas las columnas del estilo v01 muestran si los datos están validados o no. Se borran puesto que hay muy pocos no 
validados(30 de 3960 para noviembre de 2022) y además se considera que los datos importan aunque no esten validados '''

for e in df.columns:

    if 'v' in e: df = df.drop(e)
    else: pass

In [9]:
# Las columnas de la hora se cambian por la hora como id del 01 al 24

for e in df.columns:
    
    if 'h' in e: df = df.rename({e: e[1:]})
    else: pass

In [10]:
# Se quiere tener una columna con la fecha en formato Year-Month-Day hour:minute:second

# Primero se crea una columna juntando año, mes y día

fechas = []

for i in range(len(df)):

    fechas.append(f"{df.get_column('ano')[i]}-{df.get_column('mes')[i]}-{df.get_column('dia')[i]}")


df = df.with_columns(pl.Series(fechas).alias('fecha'))

df = df.drop(['ano', 'mes', 'dia'])

In [11]:
# Ahora se junta la hora con la fecha

# Se repite cada fila 24 veces

df = pl.concat([df.clone() for _ in range(24)])

In [12]:
# Se quitan los 0 de los nombres de las columnas

df.columns = [e[1:] if e[0] == '0' else e for e in df.columns]

# Se cambia el nombre de la columna 24 por 00

df = df.rename({'24': '0'})


In [13]:
# Se hace una lista con las horas para despues convertir dicha lista en una columna

horas = []

for i in range(0, len(df), 24):

    for e in range(24):

        horas.append(e)

In [14]:
df = df.with_columns(pl.Series(horas).alias('horas'))

In [15]:
# Se hace una lista con los valores de cada hora para despues convertir dicha lista en una columna

valores = []

for i in tqdm(range(0, len(df))):

    valores.append(df[i, f'{df[i, "horas"]}'])

100%|██████████| 10216296/10216296 [28:37<00:00, 5948.97it/s]


In [16]:
df = df.with_columns(pl.Series(valores).alias('valores'))

In [17]:
# Se eliminan todas las columnas de las horas que ya no interesan

df = df.drop([f'0', f'1', f'2', f'3', f'4', f'5', f'6', f'7', f'8', f'9', f'10', f'11', f'12', f'13', f'14', f'15', f'16', f'17', f'18', f'19', f'20', f'21', f'22', f'23'])

In [18]:
# Se junta la hora a la columna fecha

df = df.with_columns(pl.Series([f"{df[i, 'fecha']} {df[i, 'horas']}:00:00" for i in tqdm(range(len(df)))]).alias('fecha'))

100%|██████████| 10216296/10216296 [27:40<00:00, 6154.00it/s]


In [19]:
# Se eliminan la columna hora

df = df.drop('horas')

In [20]:
df.head()

estacion,magnitud,fecha,valores
i64,i64,str,f64
4,1,"""2022-12-1 0:00...",1.0
4,1,"""2022-12-2 1:00...",1.0
4,1,"""2022-12-3 2:00...",1.0
4,1,"""2022-12-4 3:00...",2.0
4,1,"""2022-12-5 4:00...",1.0


In [21]:
df.shape

(10216296, 4)

In [22]:
# Se guarda el df en un csv

df.write_csv(file='../final_data/pollution_data.csv')